In [ ]:
import numpy as np

### Example
* Suppose we have the following optimization problem:

$$
\begin{aligned}
    &\text{maximize} \quad x_1 + 2x_2 + 4x_3 \\
    &3x_1 + x_2 + 5x_3 \leq 10 \\
    &x_1 + 4x_2 + x_3 \leq 8 \\
    &2x_1 + 0x_2 + 2x_3 \leq 7 \\
\end{aligned}
$$

* The standard form of this optimization problem is given as:

$$
\begin{aligned}
    &\text{maximize} \quad x_1 + 2x_2 + 4x_3 + 0x_4 + 0x_ 5 + 0x_6 \\
    &3x_1 + x_2 + 5x_3 + x_4 = 10 \\
    &x_1 + 4x_2 + x_3 + x_5 = 8 \\
    &2x_1 + 0x_2 + 2x_3 + x_6 = 7 \\
\end{aligned}
$$

* We can rewrite this optimization problem as follows:

$$
\begin{aligned}
    &\text{minimize} \quad -x_1 - 2x_2 - 4x_3 - 0x_4 - 0x_ 5 - 0x_6 \\
    &3x_1 + x_2 + 5x_3 + x_4 = 10 \\
    &x_1 + 4x_2 + x_3 + x_5 = 8 \\
    &2x_1 + 0x_2 + 2x_3 + x_6 = 7 \\
\end{aligned}

$$

* We can express this optimization problem with the following matrix notation:
$$
\text{minimize} \quad \mathbf{d}^T\mathbf{x}
$$
s.t.
$$
\mathbf{A}\mathbf{x} = \mathbf{b}
$$
with $\mathbf{d} = \begin{bmatrix} -1 \\ -2 \\ -4 \\ 0 \\ 0 \\ 0 \end{bmatrix}$, $\mathbf{x} = \begin{bmatrix} x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5 \\ x_6 \end{bmatrix}$,
$\mathbf{A} = \begin{bmatrix} 3 & 1 & 5 & 1 & 0 & 0 \\ 1 & 4 & 1 & 0 & 1 & 0 \\ 2 & 0 & 2 & 0 & 0 & 1 \\ 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0  \\ 0 & 0 & 0 & 0 & 0 & 0  \end{bmatrix}$, $\mathbf{b} = \begin{bmatrix} 10 \\ 8 \\ 7 \\ 0 \\ 0 \\ 0 \end{bmatrix}$

# Implementation of ADMM
## Overview
Given the following optimization problem expressed in the standard form:
$$
\begin{aligned}
    &\text{minimize} \quad \mathbf{d}^T\mathbf{x} \\
    &\mathbf{A}\mathbf{x} = \mathbf{b} \\
\end{aligned}
$$

We can reformulate the problem by (1) introducing a new variable namely $\mathbf{y}$ and (2) introducing the constraints in the objective function:
$$
\begin{aligned}
    &\text{minimize} \quad \mathbf{d}^T\mathbf{x} + p(\mathbf{x}) + g(\mathbf{y})\\
    &\mathbf{x} = \mathbf{y} \\
\end{aligned}
$$

As a result, the ADMM steps to solve this problem are:
* Step 1: $\mathbf{x}^{\{k+1\}} = \argmin\{\}$
* Step 2: $\mathbf{y}^{\{k+1\}} = \argmin\{\}$
* Step 3: $\mu^{\{k+1\}} = \argmin\{\}$

### Step 1
In Step 1, we need to solve for $\mathbf{x}$ at the $k+1$ iteration denoted as $\mathbf{x}^{\{k+1\}}$. To do so,
* First, we need to determine the values of $\mathbf{y}$ and $\pmb{\mu}$ at the $k$ iteration denoted as $\mathbf{y}^{\{k\}}$ and $\pmb{\mu}^{\{k\}}$, respectively.
* Next, we construct the matrix $\mathbf{C}$ such that: $\mathbf{C} = \begin{bmatrix} \rho\mathbf{I} & \mathbf{A}^T \\ \mathbf{A} & 0  \end{bmatrix}$.
* Finally, we solve the following systems of linear equations to get $\mathbf{x}^{\{k+1\}}$: $\mathbf{C}\begin{bmatrix}\mathbf{x} \\ \pmb{\lambda} \end{bmatrix}= \begin{bmatrix} \rho\alpha \\ \mathbf{b} \end{bmatrix}$, with $\alpha := \mathbf{y}^{\{k\}} - \frac{1}{\rho}\Big(\mu^{\{k\}} + \mathbf{d}\Big)$.

### Step 2
In Step 2, we need to solve for $\mathbf{y}$ at the $k+1$ iteration denoted as $\mathbf{y}^{\{k+1\}}$. To do so,
* First, we need to construct $\pmb{\beta} = \mathbf{x}^{\{k+1\}} + \frac{1}{\rho}\mu^{\{k\}}$.
* Finally, we project the recently constructed vector $\pmb{\beta}$ onto the non-negative orthant to find $\mathbf{y}^{\{k+1\}}$: $\mathbf{y}^{\{k+1\}} = \Big(\pmb{\beta}\Big)_+$

### Step 3
In Step 2, we need to solve for $\pmb{\mu}$ at the $k+1$ iteration denoted as $\mu^{\{k+1\}}$. To do so,
* We simply construct $\mu^{\{k+1\}} = \mu^{\{k\}} + \rho(\mathbf{x}^{\{k+1\}} + \mathbf{y}^{\{k+1\}})$

In [ ]:
import numpy as np

def admm(d, A, b, rho=1, max_iter=100):
    """
    Implement the alternating direction method of multipliers (ADMM) optimization algorithm to solve a linear program.
    The implementation is based on the formulas mentioned in Liu et al. (2018).

    The linear program has the following form: min d^Tx, s.t. Ax = b
    """
    d = d.T
    b = b.T

    # Extract the number of variables and constraints from the matrix dimensions
    no_of_vars = A.shape[1]
    no_of_constraints = A.shape[0]

    # Initialize the dual variables y and mu
    y  = np.random.normal(size= (1,no_of_vars))
    mu = np.random.normal(size= (1,no_of_vars))

    # Construct the augmented Lagrangian matrix
    C = np.block([
        [rho * np.eye(no_of_vars), A.T],
        [A, np.zeros((no_of_constraints,no_of_constraints))]
                ])

    for i in range(max_iter):
        # Solve for the primal variable x
        alpha = y - (1/rho) * (mu + d)
        m = np.block([rho * alpha, b]).T
        x = np.linalg.solve(C, m)[: no_of_vars]

        # Apply soft-thresholding to update the dual variable y
        beta = x + (1/rho) * mu.T
        beta[beta < 0] = 0
        y = beta.T

        # Update the dual variable mu
        mu = mu + rho * (x.T - y)

    return (x,y,mu)

[[ 5.11442740e-14]
 [ 1.57894737e+00]
 [ 1.68421053e+00]
 [-1.34336986e-14]
 [ 4.99600361e-15]
 [ 3.63157895e+00]]

